In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import Libraries

In [2]:
#Install Required Libraries
# !pip install numpy pandas torch torchvision 

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder

import torch
import torchvision

# Import Dataset

In [4]:
train_df = pd.read_csv('./Data/training.csv')
test_df = pd.read_csv('./Data/test.csv')

In [5]:
# Create copies of the original DataFrames
df = train_df.copy()
df_t = test_df.copy()

# Data Exploration

In [6]:
df.head()# Display the first few rows of the dataset
print("First few rows of the dataset:")
print(df.head())

First few rows of the dataset:
   patient_id patient_race  payer_type patient_state  patient_zip3  \
0      475714          NaN    MEDICAID            CA           924   
1      349367        White  COMMERCIAL            CA           928   
2      138632        White  COMMERCIAL            TX           760   
3      617843        White  COMMERCIAL            CA           926   
4      817482          NaN  COMMERCIAL            ID           836   

   patient_age patient_gender    bmi breast_cancer_diagnosis_code  \
0           84              F    NaN                       C50919   
1           62              F  28.49                       C50411   
2           43              F  38.09                       C50112   
3           45              F    NaN                       C50212   
4           55              F    NaN                         1749   

                        breast_cancer_diagnosis_desc  ...   disabled  \
0  Malignant neoplasm of unsp site of unspecified...  ...  12

In [7]:
# Summary statistics of the numerical columns
print("\nSummary statistics of numerical columns:")
print(df.describe())


Summary statistics of numerical columns:
          patient_id  patient_zip3   patient_age          bmi    population  \
count   12906.000000  12906.000000  12906.000000  3941.000000  12905.000000   
mean   547381.196033    573.754300     59.183326    28.984539  20744.441237   
std    260404.959974    275.447534     13.335216     5.696906  13886.903756   
min    100063.000000    101.000000     18.000000    14.000000    635.545455   
25%    321517.000000    331.000000     50.000000    24.660000   9463.896552   
50%    543522.000000    554.000000     59.000000    28.190000  19154.190480   
75%    772671.750000    846.000000     67.000000    32.920000  30021.278690   
max    999896.000000    999.000000     91.000000    85.000000  71374.131580   

            density    age_median  age_under_10  age_10_to_19       age_20s  \
count  12905.000000  12905.000000  12905.000000  12905.000000  12905.000000   
mean    1581.950419     40.502259     11.122784     12.945265     13.290376   
std     2

In [8]:
# Information about the dataset, including data types and missing values
print("\nInformation about the dataset:")
print(df.info())


Information about the dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12906 entries, 0 to 12905
Data columns (total 83 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   patient_id                             12906 non-null  int64  
 1   patient_race                           6521 non-null   object 
 2   payer_type                             11103 non-null  object 
 3   patient_state                          12855 non-null  object 
 4   patient_zip3                           12906 non-null  int64  
 5   patient_age                            12906 non-null  int64  
 6   patient_gender                         12906 non-null  object 
 7   bmi                                    3941 non-null   float64
 8   breast_cancer_diagnosis_code           12906 non-null  object 
 9   breast_cancer_diagnosis_desc           12906 non-null  object 
 10  metastatic_cancer_diagnosis_code      

In [9]:
# Check for missing values in each column
print("\nMissing values in each column:")
print(df.isnull().sum())


Missing values in each column:
patient_id           0
patient_race      6385
payer_type        1803
patient_state       51
patient_zip3         0
                  ... 
veteran              1
Ozone               29
PM25                29
N02                 29
DiagPeriodL90D       0
Length: 83, dtype: int64


In [10]:
# Unique values in each column
print("\nUnique values in each column:")
for column in df.columns:
    print(f"{column}: {df[column].nunique()} unique values")


Unique values in each column:
patient_id: 12906 unique values
patient_race: 5 unique values
payer_type: 3 unique values
patient_state: 50 unique values
patient_zip3: 739 unique values
patient_age: 67 unique values
patient_gender: 1 unique values
bmi: 1342 unique values
breast_cancer_diagnosis_code: 50 unique values
breast_cancer_diagnosis_desc: 50 unique values
metastatic_cancer_diagnosis_code: 43 unique values
metastatic_first_novel_treatment: 2 unique values
metastatic_first_novel_treatment_type: 1 unique values
Region: 4 unique values
Division: 9 unique values
population: 738 unique values
density: 738 unique values
age_median: 728 unique values
age_under_10: 718 unique values
age_10_to_19: 725 unique values
age_20s: 724 unique values
age_30s: 725 unique values
age_40s: 715 unique values
age_50s: 720 unique values
age_60s: 725 unique values
age_70s: 720 unique values
age_over_80: 706 unique values
male: 722 unique values
female: 724 unique values
married: 732 unique values
divorced

## Check For Duplicates

In [11]:
# Check for duplicates in the training dataset
train_duplicates = df[df.duplicated()]

# No Duplicates Found
# print("Duplicate rows in training dataset:")
# print(train_duplicates)

# Print the shape of the training dataset before removing duplicates
print("\nShape of training dataset before removing duplicates:", df.shape)

# Remove duplicates in the training dataset and keep the first occurrence
df = df.drop_duplicates()

# Print the shape of the training dataset after removing duplicates
print("Shape of training dataset after removing duplicates:", df.shape)



Shape of training dataset before removing duplicates: (12906, 83)
Shape of training dataset after removing duplicates: (12906, 83)


# Feature Exploration and Engineering

In [37]:
selected_columns = ['patient_zip3', 'population', 'density', 'age_median', 'male', 'female']
df_temp = df[selected_columns]
print(df_temp.info())
df_temp.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12906 entries, 0 to 12905
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   patient_zip3  12906 non-null  int64  
 1   population    12905 non-null  float64
 2   density       12905 non-null  float64
 3   age_median    12905 non-null  float64
 4   male          12905 non-null  float64
 5   female        12905 non-null  float64
dtypes: float64(5), int64(1)
memory usage: 605.1 KB
None


,patient_zip3,population,density,age_median,male,female
count,12906.000000,12905.000000,12905.000000,12905.000000,12905.000000,12905.000000
mean,573.754300,20744.441237,1581.950419,40.502259,50.095180,49.904900
std,275.447534,13886.903756,2966.305306,4.036963,1.656640,1.656594
min,101.000000,635.545455,0.916667,20.600000,39.725000,38.400000
25%,331.000000,9463.896552,171.857143,37.129825,49.158333,49.062069
50%,554.000000,19154.190480,700.337500,40.639344,49.976042,50.023958
75%,846.000000,30021.278690,1666.515385,42.934783,50.937931,50.841667
max,999.000000,71374.131580,21172.000000,54.570000,61.600000,60.275000


In [13]:
c = 0
c1 = 0
for i in df_temp['breast_cancer_diagnosis_desc']:
    if 'Malig' in i:
        c+=1
    else:
        c1+=1
        
print(c)
print(c1)
# print(df_temp['breast_cancer_diagnosis_desc'][:100])

12897
9


In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

max_features=100

# Create a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=max_features, stop_words='english')

# Transform diagnosis descriptions into TF-IDF vectors
X_tfidf = tfidf_vectorizer.fit_transform(df['breast_cancer_diagnosis_desc'])

print(X_tfidf[0])

  (0, 2)	0.2373051764245159
  (0, 4)	0.2376363513440987
  (0, 26)	0.4527332302816953
  (0, 21)	0.4613661681585082
  (0, 25)	0.5516031913570816
  (0, 11)	0.26879612779577944
  (0, 10)	0.3047485189563812
